In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'seaborn'

### Load Data

In [ ]:
trainCsv = "../data/cleanLoanDataTrain.csv"
valCsv = "../data/cleanLoanDataValidation.csv"

In [ ]:
trainDf = pd.read_csv(trainCsv)
trainDf = trainDf.drop(columns=["ApplicantIncome","CoapplicantIncome"])
trainDf.head()

In [ ]:
valDf = pd.read_csv(valCsv)
valDf = valDf.drop(columns=["ApplicantIncome","CoapplicantIncome"])
valDf.head()

### Logistic Regression

In [ ]:
classifier = LogisticRegression()
classifier

In [ ]:
y = trainDf["Loan_Status"]
X = trainDf.drop(columns="Loan_Status")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

In [ ]:
classifier.fit(x_train, y_train)

In [ ]:
print(f"Training Data Score: {classifier.score(x_train, y_train)}")
print(f"Testing Data Score: {classifier.score(x_test, y_test)}")

### Logistic Regression - after log-transform
* After inspect the histogram, income and loan amount distributions are skewed
* Perform log-transformation to reduce outlier effects
* Model increases in accuracy

In [ ]:
colList = ["LoanAmount","TotalIncome"]

for col in colList: 
    fig1 = plt.figure(figsize=(12,7))
    sns.set(font_scale=1.5)
    plt.hist(trainDf[col], density=True, bins=30)
    plt.title('Histogram of ' + col)
    
    fig2 = plt.figure(figsize=(12,7))
    sns.set(font_scale=1.5)
    plt.hist(np.log(trainDf[col]), density=True, bins=30)
    plt.title('Histogram of Log-tranformed of' + col)   


In [ ]:
trainDf["Log_TotalIncome"]= np.log(trainDf["TotalIncome"])
trainDf["Log_LoanAmount"]= np.log(trainDf["LoanAmount"])

In [ ]:
y = trainDf["Loan_Status"]
X = trainDf.drop(columns=["Loan_Status","LoanAmount","TotalIncome"])

In [ ]:
classifierLog = LogisticRegression()
classifierLog

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.25)
classifierLog.fit(x_train, y_train)

In [ ]:
print(f"Training Data Score: {classifierLog.score(x_train, y_train)}")
print(f"Testing Data Score: {classifierLog.score(x_test, y_test)}")

### Logistic Regression - added features
* Uses log(total income) and log(loan amount
* Add monthly payment = loan amount/loan term
* Add remaining income = total income - monthly payment

In [ ]:
trainDf["MonthlyPayment"] = trainDf["LoanAmount"]/trainDf["Loan_Amount_Term"]
trainDf["RemainingIncome"] = trainDf["TotalIncome"] - trainDf["MonthlyPayment"]
trainDf.head()

In [ ]:
y = trainDf["Loan_Status"]
X = trainDf.drop(columns=["Loan_Status","LoanAmount","TotalIncome"])

In [ ]:
classifierAddedFea = LogisticRegression()
classifierAddedFea

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.25)
classifierAddedFea.fit(x_train, y_train)

In [ ]:
print(f"Training Data Score: {classifierAddedFea.score(x_train, y_train)}")
print(f"Testing Data Score: {classifierAddedFea.score(x_test, y_test)}")

### Save the model 
* use pickle
* use joblib

In [ ]:
valDf["Log_TotalIncome"]= np.log(valDf["TotalIncome"])
valDf["Log_LoanAmount"]= np.log(valDf["LoanAmount"])
valDf = valDf.drop(columns=["LoanAmount","TotalIncome"])

In [ ]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(classifierLog) 
  
# Load the pickled model 
classifier_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions for validation
classifier_from_pickle.predict(valDf) 

In [ ]:
import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(classifierLog, 'logisticRegression.pkl') 
  
# Load the model from the file 
classifier_from_joblib = joblib.load('logisticRegression.pkl')  
  
# Use the loaded model to make predictions 
classifier_from_joblib.predict(valDf) 

### Apply  a model on dummy data
* Choose a model that has a balance of accuracy and complexity
  - more complex models are more likely to have better accuracy, but requires more work and run time
  - simplier models are more likely to have less accuracy to the more complex models, but require much less processing and/or run time

In [ ]:
#  create dummy data to tets
colNames = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',\
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',\
       'TotalIncome']
colValues = [[1],[1],[1],[0],[0],[400000],[360],[1],[1],[60000]]
testDummyDf = pd.DataFrame(data=colValues,index=colNames).T

testDummyDf["Log_TotalIncome"]= np.log(testDummyDf["TotalIncome"])
testDummyDf["Log_LoanAmount"]= np.log(testDummyDf["LoanAmount"])
testDummyDf = testDummyDf.drop(columns=["LoanAmount","TotalIncome"])

testDummyDf

In [ ]:
# Use the loaded model to make predictions 
classifier_from_joblib.predict(testDummyDf) 